In [1]:
import os
import sys
import numpy as np
import pandas as pd

classes_dir = 'F:\Thesis_ssd\MasterThesis'
os.chdir(classes_dir)

from Classes.DataProcessing.BigDataLoader import BigDataLoader
from Classes.DataProcessing.BigDataHandler import BigDataHandler
from Classes.DataProcessing.BigDataGenerator import BigDataGenerator
from Classes.DataProcessing.BigScalerFitter import BigScalerFitter
from Classes.DataProcessing.BigStandardScalerFitter import BigStandardScalerFitter
from Classes.Modeling.Models import Models

from Classes.DataProcessing.BaselineHelperFunctions import BaselineHelperFunctions
helper = BaselineHelperFunctions()

from Classes import Tf_shutup
Tf_shutup.Tf_shutup()

from livelossplot import PlotLossesKeras

import tensorflow as tf

import keras

from keras.layers import Activation, Conv1D, Dense, Dropout, Flatten, MaxPooling3D, BatchNormalization, InputLayer, LSTM
from keras.layers import Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.losses import categorical_crossentropy
from keras.models import Sequential
from keras.utils import Sequence
from keras.optimizers import Adam
from tensorflow.keras import regularizers
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import ParameterGrid
import re
from sklearn.metrics import confusion_matrix

import random
import pprint

base_dir = 'F:\Thesis_ssd\MasterThesis'
os.chdir(base_dir)

In [2]:
data_loader = BigDataLoader(test = True, upsample = False, downsample = True, frac_diff = 0, seed = 2)

In [3]:
train, val, test = data_loader.handler.create_train_val_test(data_loader.name_label, seed = data_loader.seed)

In [4]:
data_gen = BigDataGenerator(data_loader)
train_gen = data_gen

In [5]:
############ Model picker #############
model_nr = 7

########### Hyperparameters ###########
batch_size = 256
epochs = 5
learning_rate = 0.001
#opt = tf.keras.optimizers.SGD(learning_rate=learning_rate, clipnorm=1.0, clipvalue=0.5)
opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
activation = 'tanh'
output_layer_activation = 'softmax'
dropout_rate = 0
filters = 13
kernel_size = 5
l1_r = 0.0001
l2_r = 0.01
padding = 'same'
start_neurons = 32

########### Preprocessing ###########
use_scaler = True
use_livelossplot = True

In [6]:
channels, timesteps = data_loader.handler.get_trace_shape(train)
input_shape = (batch_size, channels, timesteps)

build_model_args = {'model_nr' : model_nr,
                    'input_shape' : input_shape,
                    'num_classes' : len(np.unique(train[:,1])),
                    'dropout_rate' : dropout_rate,
                    'activation' : activation,
                    'output_layer_activation' : output_layer_activation,
                    'l2_r' : l2_r,
                    'l1_r' : l1_r,
                    'full_regularizer' : True,
                    'start_neurons' : start_neurons,
                    'filters' : filters,
                    'kernel_size' : kernel_size,
                    'padding' : 'same'}
model = Models(**build_model_args).model

model_args = {'loss' : "binary_crossentropy",
              'optimizer' : opt,
              'metrics' : ["accuracy","MSE",
                           tf.keras.metrics.Precision(thresholds=None, top_k=None, class_id=None, name=None, dtype=None),
                           tf.keras.metrics.Recall(thresholds=None, top_k=None, class_id=None, name=None, dtype=None)]}
model.compile(**model_args)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (256, 3, 32)              772224    
_________________________________________________________________
dropout (Dropout)            (256, 3, 32)              0         
_________________________________________________________________
batch_normalization (BatchNo (256, 3, 32)              128       
_________________________________________________________________
lstm_1 (LSTM)                (256, 16)                 3136      
_________________________________________________________________
dropout_1 (Dropout)          (256, 16)                 0         
_________________________________________________________________
batch_normalization_1 (Batch (256, 16)                 64        
_________________________________________________________________
flatten (Flatten)            (256, 16)                 0

In [7]:
scaler = None
if use_scaler:
    scaler = BigStandardScalerFitter(train, data_loader.handler).fit_scaler()    
    

gen_args = {
    'batch_size' : batch_size,
    'use_scaler' : use_scaler,
    'scaler' : scaler
}


train_gen = data_gen.data_generator(train, **gen_args)
val_gen = data_gen.data_generator(val, **gen_args)
test_gen = data_gen.data_generator(test, **gen_args)


callbacks = PlotLossesKeras()

args = {'steps_per_epoch' : helper.get_steps_per_epoch(train, batch_size, False),
        'epochs' : epochs,
        'validation_data' : val_gen,
        'validation_steps' : helper.get_steps_per_epoch(val, batch_size, False),
        'verbose' : 1,
        'use_multiprocessing' : False, 
        'workers' : 1,
        'callbacks' : [callbacks]
}

model_fit = model.fit(train_gen, **args)

C:\Users\tss_9\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


Epoch 1/5
32/32 [============================>.] - ETA: 6s - loss: 4.2478 - accuracy: 0.5380 - MSE: 0.2621 - precision: 0.5380 - recall: 0.5380 

UnknownError:  NameError: name 'random' is not defined
Traceback (most recent call last):

  File "C:\Users\tss_9\anaconda3\lib\site-packages\tensorflow\python\ops\script_ops.py", line 244, in __call__
    ret = func(*args)

  File "C:\Users\tss_9\anaconda3\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 302, in wrapper
    return func(*args, **kwargs)

  File "C:\Users\tss_9\anaconda3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 827, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "C:\Users\tss_9\anaconda3\lib\site-packages\tensorflow\python\keras\engine\data_adapter.py", line 814, in wrapped_generator
    for data in generator_fn():

  File "F:\Thesis_ssd\MasterThesis\Classes\DataProcessing\BigDataGenerator.py", line 29, in data_generator
    i_start = random.randint(0, num_samples-overflow)

NameError: name 'random' is not defined


	 [[{{node PyFunc}}]]
	 [[IteratorGetNext]] [Op:__inference_train_funct